In [1]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [38]:
print("hello world")

hello world


# libs

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision

import timm

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sys
from tqdm.notebook import tqdm

In [4]:
from PIL import Image  # Importing the Image module
import numpy as np
import matplotlib.pyplot as plt

# hyper-param

In [5]:
##########################
### SETTINGS
##########################

# Hyperparameters
RANDOM_SEED = 1

# Dynamically set device
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {DEVICE}")

GRAYSCALE = False

Using device: cuda


In [6]:
train_size = int(10000*.8)
validation_size = int(10000*.1)
test_size = int(10000*.1)

### model development and fine-tuning

In [7]:
import torchvision.models as models

# Load the pre-trained ResNet-18 model
# model = models.resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 175MB/s] 


In [8]:
# Freeze all the pre-trained layers
for param in model.parameters():
    param.requires_grad = False

In [9]:
# Modify the last layer of the model
num_classes = 2 # replace with the number of classes in your dataset
model.fc = torch.nn.Linear(model.fc.in_features, num_classes)

In [10]:
for param in model.fc.parameters():
    param.requires_grad = True

### loading dataset 

In [11]:

from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from torchvision import datasets
from torchvision import transforms

In [12]:
#The target label is attractiveness and protected attribute is gender.

data_root = "/kaggle/input/celeba-dataset/"
label_attr = "Attractive"
protected_attr = "Male"

In [13]:
class CelebaDataset(Dataset):
    
    def __init__(self, csv_path, img_dir, transform=None, label_attr = 'Attractive', protected_attr = 'Male'):
        
        df = pd.read_csv(csv_path, index_col=0)
        self.img_dir = img_dir
        self.csv_path = csv_path
        self.img_names = df.index.values
        self.y = df[label_attr].values
        self.p = df[protected_attr].values
        self.transform = transform
        
    def __getitem__(self, index):
        img = Image.open(os.path.join(self.img_dir,
                      self.img_names[index]))

        if self.transform is not None:
            img = self.transform(img)
            
        label = self.y[index]
        protected = self.p[index]
        return img, label, protected
        
    def __len__(self):
        return self.y.shape[0]


In [14]:
other_data_root = '/kaggle/working/' #for writing purposes

In [15]:
BATCH_SIZE=1500

In [16]:
def prepare_dataset(label_attr = 'Attractive', protected_attr = 'Male'):

    print('Using label attribute:', label_attr)

    df1 = pd.read_csv(data_root + 'list_attr_celeba.csv',usecols=[label_attr, protected_attr])
    # Make 0 (female) & 1 (male) labels instead of -1 & 1
    df1.loc[df1[label_attr] == -1, label_attr] = 0
    df1.loc[df1[protected_attr] == -1, protected_attr] = 0
    
#     print(f"df1\n{df1}")

    df2 = pd.read_csv(data_root + 'list_eval_partition.csv')
    df2.columns = ['image_id', 'partition']
    image_series = df2['image_id']
    df2 = df2.set_index('image_id')
#     print(f"df2\n{df2}")

#     print("\n\n")
#     print(f"df2 shape {df2.shape} and df1 shape {df1.shape}")

    df1 = df1.set_index(image_series)

#     df3 = df1.merge(df2, left_index=True, right_index=True)
    df3 = pd.concat([df1,df2],axis=1)
#     print("\n\n\n\n")
#     print(df1.index)  
#     print(df2.index)
#     print("\n\n\n\n")
    
#     print(f"alt df3 \n{ df1.merge(df2, left_index=True, right_index=True)}")
#     print(f"df3\n{df3}")

    
    
    
    df3.to_csv(other_data_root + 'celeba-attractive-partitions.csv')
#     print(df3)
#     print(pd.read_csv(new_data_root+'celeba-attractive-partitions.csv'))
    df4 = pd.read_csv(other_data_root + 'celeba-attractive-partitions.csv', index_col=0)

    df4.loc[df4['partition'] == 0].to_csv(other_data_root + 'celeba-attractive-train.csv')
    df4.loc[df4['partition'] == 1].to_csv(other_data_root + 'celeba-attractive-valid.csv')
    df4.loc[df4['partition'] == 2].to_csv(other_data_root + 'celeba-attractive-test.csv')
    
#     print(f"Train samples: {df4.loc[df4['partition'] == 0].shape[0]}")
#     print(f"Valid samples: {df4.loc[df4['partition'] == 1].shape[0]}")
#     print(f"Test samples: {df4.loc[df4['partition'] == 2].shape[0]}")

    
def get_loaders():

    prepare_dataset(label_attr, protected_attr)
    
    # Note that transforms.ToTensor()
    # already divides pixels by 255. internally

    custom_transform = transforms.Compose([transforms.CenterCrop((178, 178)),
                                           transforms.Resize((128, 128)),
                                           transforms.ToTensor()])

    train_dataset = CelebaDataset(csv_path=other_data_root + 'celeba-attractive-train.csv',
                                  img_dir=data_root + 'img_align_celeba/img_align_celeba',
                                  transform=custom_transform,
                                  label_attr=label_attr,
                                  protected_attr = protected_attr)

    valid_dataset = CelebaDataset(csv_path=other_data_root + 'celeba-attractive-valid.csv',
                                  img_dir=data_root + 'img_align_celeba/img_align_celeba',
                                  transform=custom_transform,
                                  label_attr=label_attr,
                                  protected_attr = protected_attr)

    test_dataset = CelebaDataset(csv_path=other_data_root + 'celeba-attractive-test.csv',
                                 img_dir=data_root + 'img_align_celeba/img_align_celeba',
                                 transform=custom_transform,
                                 label_attr=label_attr,
                                 protected_attr = protected_attr)

    train_loader = DataLoader(dataset=train_dataset,
                              batch_size=BATCH_SIZE,
                              shuffle=True,
                              num_workers=4)

    valid_loader = DataLoader(dataset=valid_dataset,
                              batch_size=BATCH_SIZE,
                              shuffle=False,
                              num_workers=4)
    
    test_loader = DataLoader(dataset=test_dataset,
                             batch_size=BATCH_SIZE,
                             shuffle=False,
                             num_workers=4)

    return train_loader, valid_loader, test_loader


In [17]:
train_loader, valid_loader, test_loader = get_loaders()

Using label attribute: Attractive


In [18]:
# for i in rangetest_loader:

In [19]:
import os

In [20]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.cuda.amp import autocast, GradScaler
import time

In [21]:
# Define the criterion, optimizer, and other parameters
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
scaler = GradScaler()  # For mixed precision training

num_epochs = 25
validate_every = 2
patience = 3  # Number of epochs to wait before early stopping
best_valid_loss = float('inf')
no_improve_epochs = 0

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
print("heu")

heu


/tmp/ipykernel_30/309015131.py:4: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()  # For mixed precision training


In [22]:
# Training and validation loop
for epoch in range(num_epochs):
    start_time = time.time()  # Track time at the start of each epoch
    model.train()  # Set model to training mode
    running_loss = 0.0  # Reset cumulative training loss for this epoch

    print(f"Epoch {epoch + 1}/{num_epochs}: Starting training...")
    for batch_idx, (images, labels, _) in enumerate(train_loader):  # Loop through training batches
        images, labels = images.to(device), labels.to(device)  # Move data to the selected device
        optimizer.zero_grad()  # Clear gradients from the previous step

        # Forward pass with mixed precision for efficiency
        with autocast():
            outputs = model(images)  # Get predictions from the model
            loss = criterion(outputs, labels)  # Calculate the loss

        # Backward pass and optimization step
        scaler.scale(loss).backward()  # Scale the loss for mixed precision
        scaler.step(optimizer)  # Update model weights
        scaler.update()  # Update the scaler for mixed precision

        running_loss += loss.item()  # Accumulate training loss

        # Optional: Log batch-level progress
        if (batch_idx + 1) % (len(train_loader) // 5) == 0:  # Log 5 times per epoch
            print(f"  Batch [{batch_idx + 1}/{len(train_loader)}], Loss: {loss.item():.4f}")

    # Calculate average training loss for the epoch
    avg_train_loss = running_loss / len(train_loader)
    epoch_time = time.time() - start_time  # Calculate epoch duration
    print(f"Epoch [{epoch + 1}/{num_epochs}] Complete. Train Loss: {avg_train_loss:.4f}, Time: {epoch_time:.2f}s")

    # Validation step (only every `validate_every` epochs)
    if (epoch + 1) % validate_every == 0:
        print(f"Epoch {epoch + 1}: Starting validation...")
        model.eval()  # Set model to evaluation mode
        valid_loss = 0.0  # Reset cumulative validation loss
        correct = 0  # Count of correct predictions
        total = 0  # Total number of samples

        with torch.no_grad():  # No gradients needed during validation
            for images, labels, _ in valid_loader:  # Loop through validation batches
                images, labels = images.to(device), labels.to(device)  # Move data to the selected device

                outputs = model(images)  # Get predictions
                loss = criterion(outputs, labels)  # Calculate the loss
                valid_loss += loss.item()  # Accumulate validation loss

                # Calculate accuracy
                _, predicted = torch.max(outputs, 1)  # Get predicted class
                total += labels.size(0)  # Update total sample count
                correct += (predicted == labels).sum().item()  # Update correct prediction count

        # Calculate validation loss and accuracy
        avg_valid_loss = valid_loss / len(valid_loader)
        accuracy = 100 * correct / total
        print(f"Validation Results - Loss: {avg_valid_loss:.4f}, Accuracy: {accuracy:.2f}%")

        # Early stopping logic
        if avg_valid_loss < best_valid_loss:
            best_valid_loss = avg_valid_loss  # Update the best validation loss
            no_improve_epochs = 0  # Reset patience counter
            print("  Validation loss improved. Resetting early stopping counter.")
        else:
            no_improve_epochs += 1  # Increment patience counter
            print(f"  No improvement in validation loss. Patience: {no_improve_epochs}/{patience}")
            if no_improve_epochs >= patience:
                print("Early stopping triggered. Stopping training.")
                break  # Exit the training loop
    

Epoch 1/25: Starting training...


/tmp/ipykernel_30/70649097.py:13: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


  Batch [21/109], Loss: 0.7499
  Batch [42/109], Loss: 0.7051
  Batch [63/109], Loss: 0.6557
  Batch [84/109], Loss: 0.6319
  Batch [105/109], Loss: 0.6174
Epoch [1/25] Complete. Train Loss: 0.6776, Time: 360.62s
Epoch 2/25: Starting training...
  Batch [21/109], Loss: 0.6003
  Batch [42/109], Loss: 0.5612
  Batch [63/109], Loss: 0.5489
  Batch [84/109], Loss: 0.5515
  Batch [105/109], Loss: 0.5315
Epoch [2/25] Complete. Train Loss: 0.5669, Time: 137.13s
Epoch 2: Starting validation...
Validation Results - Loss: 0.5596, Accuracy: 71.61%
  Validation loss improved. Resetting early stopping counter.
Epoch 3/25: Starting training...
  Batch [21/109], Loss: 0.5263
  Batch [42/109], Loss: 0.5231
  Batch [63/109], Loss: 0.5329
  Batch [84/109], Loss: 0.5307
  Batch [105/109], Loss: 0.5132
Epoch [3/25] Complete. Train Loss: 0.5324, Time: 138.25s
Epoch 4/25: Starting training...
  Batch [21/109], Loss: 0.5255
  Batch [42/109], Loss: 0.5127
  Batch [63/109], Loss: 0.5163
  Batch [84/109], Loss:

In [33]:
# Training and validation loop
for epoch in range(num_epochs):
    start_time = time.time()  # Track time at the start of each epoch
    model.train()  # Set model to training mode
    running_loss = 0.0  # Reset cumulative training loss for this epoch

    print(f"Epoch {epoch + 1}/{num_epochs}: Starting training...")
    for batch_idx, (images, labels, _) in enumerate(train_loader):  # Loop through training batches
        images, labels = images.to(device), labels.to(device)  # Move data to the selected device
        optimizer.zero_grad()  # Clear gradients from the previous step

        # Forward pass with mixed precision for efficiency
        with autocast():
            outputs = model(images)  # Get predictions from the model
            loss = criterion(outputs, labels)  # Calculate the loss

        # Backward pass and optimization step
        scaler.scale(loss).backward()  # Scale the loss for mixed precision
        scaler.step(optimizer)  # Update model weights
        scaler.update()  # Update the scaler for mixed precision

        running_loss += loss.item()  # Accumulate training loss

        # Optional: Log batch-level progress
        if (batch_idx + 1) % (len(train_loader) // 5) == 0:  # Log 5 times per epoch
            print(f"  Batch [{batch_idx + 1}/{len(train_loader)}], Loss: {loss.item():.4f}")

    # Calculate average training loss for the epoch
    avg_train_loss = running_loss / len(train_loader)
    epoch_time = time.time() - start_time  # Calculate epoch duration
    print(f"Epoch [{epoch + 1}/{num_epochs}] Complete. Train Loss: {avg_train_loss:.4f}, Time: {epoch_time:.2f}s")

    # Validation step (only every `validate_every` epochs)
    if (epoch + 1) % validate_every == 0:
        print(f"Epoch {epoch + 1}: Starting validation...")
        model.eval()  # Set model to evaluation mode
        valid_loss = 0.0  # Reset cumulative validation loss
        correct = 0  # Count of correct predictions
        total = 0  # Total number of samples

        with torch.no_grad():  # No gradients needed during validation
            for images, labels, _ in valid_loader:  # Loop through validation batches
                images, labels = images.to(device), labels.to(device)  # Move data to the selected device

                outputs = model(images)  # Get predictions
                loss = criterion(outputs, labels)  # Calculate the loss
                valid_loss += loss.item()  # Accumulate validation loss

                # Calculate accuracy
                _, predicted = torch.max(outputs, 1)  # Get predicted class
                total += labels.size(0)  # Update total sample count
                correct += (predicted == labels).sum().item()  # Update correct prediction count

        # Calculate validation loss and accuracy
        avg_valid_loss = valid_loss / len(valid_loader)
        accuracy = 100 * correct / total
        print(f"Validation Results - Loss: {avg_valid_loss:.4f}, Accuracy: {accuracy:.2f}%")

        # Early stopping logic
        if avg_valid_loss < best_valid_loss:
            best_valid_loss = avg_valid_loss  # Update the best validation loss
            no_improve_epochs = 0  # Reset patience counter
            print("  Validation loss improved. Resetting early stopping counter.")
        else:
            no_improve_epochs += 1  # Increment patience counter
            print(f"  No improvement in validation loss. Patience: {no_improve_epochs}/{patience}")
            if no_improve_epochs >= patience:
                print("Early stopping triggered. Stopping training.")
                break  # Exit the training loop


Epoch 1/10: Starting training...


/tmp/ipykernel_30/70649097.py:13: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


  Batch [21/109], Loss: 0.4262
  Batch [42/109], Loss: 0.3906
  Batch [63/109], Loss: 0.3635
  Batch [84/109], Loss: 0.3514
  Batch [105/109], Loss: 0.3698
Epoch [1/10] Complete. Train Loss: 0.4130, Time: 152.43s
Epoch 2/10: Starting training...
  Batch [21/109], Loss: 0.3406
  Batch [42/109], Loss: 0.3388
  Batch [63/109], Loss: 0.3426
  Batch [84/109], Loss: 0.3607
  Batch [105/109], Loss: 0.3479
Epoch [2/10] Complete. Train Loss: 0.3491, Time: 147.53s
Epoch 2: Starting validation...
Validation Results - Loss: 0.5692, Accuracy: 78.66%
  No improvement in validation loss. Patience: 1/3
Epoch 3/10: Starting training...
  Batch [21/109], Loss: 0.3239
  Batch [42/109], Loss: 0.3331
  Batch [63/109], Loss: 0.3528
  Batch [84/109], Loss: 0.3566
  Batch [105/109], Loss: 0.3494
Epoch [3/10] Complete. Train Loss: 0.3333, Time: 146.63s
Epoch 4/10: Starting training...
  Batch [21/109], Loss: 0.2820
  Batch [42/109], Loss: 0.2974
  Batch [63/109], Loss: 0.3257
  Batch [84/109], Loss: 0.3102
  B

In [35]:
num_epochs = 10
optimizer = optim.Adam(model.parameters(), lr=0.00001)
# Training and validation loop
for epoch in range(num_epochs):
    start_time = time.time()  # Track time at the start of each epoch
    model.train()  # Set model to training mode
    running_loss = 0.0  # Reset cumulative training loss for this epoch

    print(f"Epoch {epoch + 1}/{num_epochs}: Starting training...")
    for batch_idx, (images, labels, _) in enumerate(train_loader):  # Loop through training batches
        images, labels = images.to(device), labels.to(device)  # Move data to the selected device
        optimizer.zero_grad()  # Clear gradients from the previous step

        with autocast():
            outputs = model(images)  
            loss = criterion(outputs, labels) 

        scaler.scale(loss).backward()  
        scaler.step(optimizer)  
        scaler.update()  

        running_loss += loss.item()  

        if (batch_idx + 1) % (len(train_loader) // 5) == 0:  
            print(f"  Batch [{batch_idx + 1}/{len(train_loader)}], Loss: {loss.item():.4f}")

    avg_train_loss = running_loss / len(train_loader)
    epoch_time = time.time() - start_time  
    print(f"Epoch [{epoch + 1}/{num_epochs}] Complete. Train Loss: {avg_train_loss:.4f}, Time: {epoch_time:.2f}s")

    if (epoch + 1) % validate_every == 0:
        print(f"Epoch {epoch + 1}: Starting validation...")
        model.eval()  
        valid_loss = 0.0  
        correct = 0  
        total = 0  

        with torch.no_grad():  
            for images, labels, _ in valid_loader: 
                images, labels = images.to(device), labels.to(device)  
                outputs = model(images)  # Get predictions
                loss = criterion(outputs, labels)  # Calculate the loss
                valid_loss += loss.item()  # Accumulate validation loss

               
                _, predicted = torch.max(outputs, 1)  
                total += labels.size(0)  
                correct += (predicted == labels).sum().item()  

       
        avg_valid_loss = valid_loss / len(valid_loader)
        accuracy = 100 * correct / total
        print(f"Validation Results - Loss: {avg_valid_loss:.4f}, Accuracy: {accuracy:.2f}%")

        
        if avg_valid_loss < best_valid_loss:
            best_valid_loss = avg_valid_loss  # Update the best validation loss
            no_improve_epochs = 0  # Reset patience counter
            print("  Validation loss improved. Resetting early stopping counter.")
        else:
            no_improve_epochs += 1  # Increment patience counter
            print(f"  No improvement in validation loss. Patience: {no_improve_epochs}/{patience}")
            if no_improve_epochs >= patience:
                print("Early stopping triggered. Stopping training.")
                break  # Exit the training loop
    

Epoch 1/10: Starting training...


/tmp/ipykernel_30/2374052875.py:15: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


  Batch [21/109], Loss: 0.0453
  Batch [42/109], Loss: 0.0497
  Batch [63/109], Loss: 0.0424
  Batch [84/109], Loss: 0.0412
  Batch [105/109], Loss: 0.0443
Epoch [1/10] Complete. Train Loss: 0.0431, Time: 149.14s
Epoch 2/10: Starting training...
  Batch [21/109], Loss: 0.0372
  Batch [42/109], Loss: 0.0295
  Batch [63/109], Loss: 0.0324
  Batch [84/109], Loss: 0.0329
  Batch [105/109], Loss: 0.0343
Epoch [2/10] Complete. Train Loss: 0.0274, Time: 150.97s
Epoch 2: Starting validation...
Validation Results - Loss: 0.9119, Accuracy: 79.59%
  No improvement in validation loss. Patience: 4/3
Early stopping triggered. Stopping training.


In [36]:
model.eval()
test_loss = 0.0
correct = 0
total = 0

with torch.no_grad():
    for images, labels, _ in test_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)
        test_loss += loss.item()

        # Accuracy calculation
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

avg_test_loss = test_loss / len(test_loader)
test_accuracy = 100 * correct / total
print(f"Test Loss: {avg_test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%")

Test Loss: 0.8629, Test Accuracy: 80.06%


In [37]:
torch.save(model.state_dict(), "/kaggle/working/fine_tuned_best_ever_resnet18")